In [1]:
# RAG (Retrieval Augmented Generation) — это метод работы с большими
# языковыми моделями, когда пользователь пишет свой вопросы, а программно к этому
# вопросу «подмешивается» дополнительная информация из каких-то внешних
# источников и подается все целиком на вход языковой модели. Другими словами,
# добавляется в контекст запроса к языковой модели дополнительная информация, на
# основе которой языковая модель может дать пользователю более полный и точный
# ответ. По сути три компоненты: Retrieval, Augmented, Generation
CLIENT_AUTH = ''

In [40]:
pip install langchain_text_splitters langchain_community chroma gigachat langchain chromadb pypdf

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement re (from versions: none)
ERROR: No matching distribution found for re


In [28]:
# Начнем с Retrieval:
# Чтобы сформировать векторную базу данных (Chroma/FAISS) необходимы документы с данными, которые мы векторизуем, 
# с помощью функции встраивания (embbeding)

from langchain_community.embeddings.gigachat import GigaChatEmbeddings

embedding = GigaChatEmbeddings(
    credentials=CLIENT_AUTH, 
    scope="GIGACHAT_API_PERS",
    verify_ssl_certs=False
)

In [29]:
# Необходимо обработать данные, а именно выбрать их формат, разделить на чанки и загрузить в векторную базу данных Chroma
# Делим документы на чанки:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.schema import Document

def split_docs(documents: list[Document]):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=650,
        chunk_overlap=150,
        length_function=len,
        is_separator_regex=False
    )
    return text_splitter.split_documents(documents)

In [30]:
# Необходимо выбрать загрузчик документов (PDF, DOXC), загрузка происходит из папки /data
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.document_loaders import PyPDFDirectoryLoader

def PDFload_docs():
    documents_loader = PyPDFDirectoryLoader('data/')
    return documents_loader.load()

# Можно использовать WebBaseLoader для загрузки из ссылок

In [31]:
#  Определяем векторную БД, в нашем случае Chroma 
from langchain_community.vectorstores.chroma import Chroma

CHROMA_PATH = 'chroma'

db = Chroma(
        persist_directory=CHROMA_PATH,
        embedding_function=embedding,
        collection_metadata={"hnsw:space": "cosine"}
        )

In [32]:
# Функция расчета id чанка
from typing import List

def calculate_chunk_ids(chunks):
    # Формат: "data/Adobe....pdf:6:2"
    # Page Source : Page Number : Chunk Index
    last_page_id = None
    current_chunk_index = 0
    for chunk in chunks:
        source = chunk.metadata.get("source")
        page = chunk.metadata.get("page")
        current_page_id = f"{source}:{page}"
    # Если идентификатор страницы совпадает с предыдущим, увеличьте индекс.
        if current_page_id == last_page_id:
            current_chunk_index += 1
        else:
            current_chunk_index = 0
        # Вычисление id для чанка
        chunk_id = f"{current_page_id}:{current_chunk_index}"
        last_page_id = current_page_id
        # Добавьте его в метаданные страницы.
        chunk.metadata["id"] = chunk_id
    return chunks

 # Функция добавления чанков в Chroma 

def add_to_chroma(chunks: List[Document]):
    # embedding = get_embedding_func()
    db = Chroma(
        persist_directory=CHROMA_PATH,
        embedding_function=embedding,
        collection_metadata={"hnsw:space": "cosine"}
        )
    # Вычисляем идентификаторы для каждого чанка (части документа).
    chunks_with_ids = calculate_chunk_ids(chunks)

    # Получаем существующие документы из базы данных.
    existing_items = db.get(
        include=[])  # Получаем только идентификаторы документов, так как они всегда включены по умолчанию
    existing_ids = set(existing_items["ids"])  # Преобразуем список идентификаторов в множество для быстрого поиска
    print(f"Number of existing documents in DB: {len(existing_ids)}")

    # Отбираем только те документы, которых еще нет в базе данных.
    new_chunks = []
    for chunk in chunks_with_ids:
        if chunk.metadata["id"] not in existing_ids:
            new_chunks.append(chunk)

    if len(new_chunks):
        print(f"👉 ^_^ Adding new documents: {len(new_chunks)}")
        new_chunk_ids = [chunk.metadata["id"] for chunk in new_chunks]

        # Обрабатываем документы пакетами по 166 штук для оптимизации производительности.
        batch_size = 166
        for i in range(0, len(new_chunks), batch_size):
            batch_chunks = new_chunks[i:i + batch_size]
            batch_ids = new_chunk_ids[i:i + batch_size]
            db.add_documents(batch_chunks,
                             ids=batch_ids)  # Добавляем пакет документов в базу данных с указанием их идентификаторов

        db.persist()  # Сохраняем изменения в базе данных на диск
    else:
        print("✅ No new documents to add")

In [33]:
# Опишем процесс создания БД
documents = PDFload_docs()
chunks = split_docs(documents)
add_to_chroma(chunks)

Number of existing documents in DB: 0
👉 ^_^ Adding new documents: 8


C:\Users\dan_p\AppData\Local\Temp\ipykernel_37708\1912922927.py:61: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  db.persist()  # Сохраняем изменения в базе данных на диск


In [34]:
# Имеем следующий промпт:
PROMPT_TEMPLATE = """
Ответьте на вопрос, основываясь только на следующем контексте:

{context}

---

Ответьте на вопрос, базируясь только на приведенном выше контексте: {question}

Если в контексте нет ответа на вопрос, то ответь, что не знаешь о чем идет речь.
В ответе не используй слово контекст.
"""

In [38]:
# Напиши вопрос пользователя к ai-помощнику:
query_text = 'Какие способы автомотизировать бизнес'

In [43]:
# Когда мы загрузили данные в векторную бд, определив, как показано выше. Достаем из бд 5 наиболее релевантных чанка, с помощью:
from langchain.prompts import ChatPromptTemplate

db = Chroma(
        persist_directory=CHROMA_PATH,
        embedding_function=embedding,
        collection_metadata={"hnsw:space": "cosine"}
    )

results = db.similarity_search_with_relevance_scores(query_text, k=3)

# Далее уже используем промпт выше для подмешивания контекста 
context_text = "\n---\n".join([doc.page_content for doc, _score in results])
prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
prompt = prompt_template.format(context=context_text, question=query_text)

print(prompt)

Human: 
Ответьте на вопрос, основываясь только на следующем контексте:

•География — предпочтительно по городам и между городами РФ.
•Основной транспорт — малотоннажные, среднетоннажные и крупнотоннажные 
грузовики.
•Деятельность соответствует ОКВЭД 49.4 «Деятельность автомобильного грузового 
транспорта и услуги по перевозкам».
Рассмотрим клиента, который крутит баранку уже 25 лет, а 10 лет назад открыл свою 
транспортную компанию «Гудок».
Какие есть способы автомотизировать бизнес:
Как это работает сейчас: все данные по заказам клиента записывает в тетрадь.
Что предлагаем мы: база заказов, водителей и транспортных средств доступна онлайн. Кроме
---
будет формироваться персональная аналитика по эффективности работы автопарка и 
оплаченным заказам.
Автоматизировать бизнес будет не так сложно, если обратится за помощью к нам, ведь у нас 
есть готовое решение —« СберБизнес. Транспорт» .
«СберБизнес. Транспорт»  доступен в 3 форматах:
Веб-версия для владельца
•Аналитика по доходам и заказ

In [47]:
from langchain_community.chat_models import GigaChat
# Берем модель, засовываем в нее промпт и invoke()
# Теперь вы умеете писать ai-помощника
messages = []

model = GigaChat(
        credentials=CLIENT_AUTH,
        # scope="GIGACHAT_API_PERS",
        model="GigaChat",
        verify_ssl_certs=False
    )

messages.append({"role": "user", "content": prompt})
response_text = model.invoke(messages)

In [48]:
print(response_text)

content='Клиент может автоматизировать свой бизнес, используя систему "СберБизнес. Транспорт". Эта система предоставляет веб-версию для владельца, которая включает аналитику по доходам и заказам, актуальные статусы по оплате счетов и задолженностям, базу данных транспортных средств и водителей, а также учет расходов на содержание автопарка. Она позволяет владельцу бизнеса иметь все данные под рукой 24/7, автоматизировать рутинные процессы и формировать первичные документы в один клик. Также система предоставляет возможность быстро и удобно передавать водителям информацию по заказам, анализировать эффективность работы автопарка и сравнивать ее с конкурентами, а также экономить за счет специальных предложений внутри отраслевого решения.' additional_kwargs={} response_metadata={'token_usage': Usage(prompt_tokens=546, completion_tokens=158, total_tokens=704), 'model_name': 'GigaChat:1.0.26.15', 'finish_reason': 'stop'} id='run-dc5a6d1d-fc13-483a-8c84-01935a73a3f5-0'
